<center><span style="font-size:30px; font-weight: bold;">GIS Final Report</span></center>
<center><span style="font-size:24px;">Analysis of greenhouse gas emissions (GHG) in Texas</span></center>

<center><span style="font-size:22px;"><b>Supplement 1:</b> Data cleaning </span></center>

## Introduction to this section

The two Climate Trace datasets from 2022 and 2023 are too large to open in Excel, so they require filtering. Each shows company/source emissions each month over a period of up to 12 months. For every company, there are twelve months' emissions data for multiple gases. I decide to focus only on methane (CH4), carbon dioxide (CO2), nitrous oxides (NOx), and fine particulate matter (PM2.5). All other emissions are filtered out. I then calculate the average monthly emissions by calculating the mean emissions over 12 months for each company/gas combination. I then drop any data with emissions of zero (they are assumed to be unreported) and save the results for each gas in separate tables. These are then loaded into ArcGIS.

Later, I create an emissions comparison table to compare the results for each company in 2023 and 2022, measuring the absolute and percentage change in emissions. Where a company appears in one database but not another, I decide to drop that company from the dataset (using the equivalent of an inner join).

Finally, I run a validation to confirm that the pipeline is correctly filtering the results from the 2022 and 2023 data. I find suspicious reporting from some companies, which will be discussed further in my final report.

## Imports

In [127]:
import geopandas as gpd
import pandas as pd
import numpy as np
import duckdb as ddb
import os
import json

pd.set_option("display.max_columns", None)
pd.options.display.float_format = "{:.5f}".format

In [128]:
base_folder = r"C:\Users\james\OneDrive - Högskolan Dalarna\01 - Turing College\D99 - Capstone Project\Methane emissions - Texas"

climate_trace_TX_2023_csv = os.path.join(
    base_folder, "Climate Trace", "Texas_USA_climate_trace_2023.csv"
)

climate_trace_TX_2022_csv = os.path.join(
    base_folder, "Climate Trace", "Texas_USA_climate_trace_2022.csv"
)

# Pipeline

To achieve the aims above, I create functions for each stage of the process and run them through a pipeline. This makes it easier to apply the results to both the 2022 and 2023 datasets without the potential for human error.

In [129]:
import pandas as pd


class ClimateDataPipeline:
    def __init__(self, file_path: str) -> None:
        """Initializes the pipeline with a dataset file path.

        Args:
            file_path (str): Path to the dataset CSV file.
        """
        self.file_path: str = file_path
        self.df: pd.DataFrame | None = None  # Placeholder for DataFrame

    def load_data(self) -> "ClimateDataPipeline":
        """Loads the dataset into a Pandas DataFrame.

        Returns:
            ClimateDataPipeline: The pipeline instance with loaded data.
        """
        self.df = pd.read_csv(self.file_path)
        return self

    def convert_dates(self) -> "ClimateDataPipeline":
        """Converts 'end_time' to datetime and extracts year & month.

        Returns:
            ClimateDataPipeline: The pipeline instance with transformed date data.
        """
        if self.df is not None and "end_time" in self.df.columns:
            self.df["end_time"] = pd.to_datetime(self.df["end_time"], errors="coerce")
            self.df["end_time"] = self.df["end_time"].dt.strftime(
                "%m-%Y"
            )  # Month-Year format
            self.df["year"] = self.df["end_time"].str[-4:]  # Extract year
        return self

    def filter_gases(
        self, gases_of_interest: list[str] | None = None
    ) -> "ClimateDataPipeline":
        """Filters the dataset for specific gases.

        Args:
            gases_of_interest (list[str] | None): List of gases to filter. Defaults to common gases.

        Returns:
            ClimateDataPipeline: The pipeline instance with filtered data.
        """
        if gases_of_interest is None:
            gases_of_interest = ["ch4", "co2", "nox", "pm2_5"]

        if self.df is not None and "gas" in self.df.columns:
            self.df = self.df[self.df["gas"].isin(gases_of_interest)]
        return self

    def calculate_avg_emissions(self) -> "ClimateDataPipeline":
        """Calculates average monthly emissions per source & gas type.

        Returns:
            ClimateDataPipeline: The pipeline instance with calculated averages.
        """
        if self.df is not None and {"emissions_quantity", "source_id", "gas"}.issubset(
            self.df.columns
        ):
            self.df["average_monthly_emissions_tons"] = self.df.groupby(
                ["source_id", "gas"]
            )["emissions_quantity"].transform("mean")
        return self

    def drop_zero_emissions(self) -> "ClimateDataPipeline":
        """Drops rows where monthly average emissions are 0.

        Returns:
            ClimateDataPipeline: The pipeline instance with zero-emission rows removed.
        """
        if self.df is not None and "average_monthly_emissions_tons" in self.df.columns:
            self.df = self.df[self.df["average_monthly_emissions_tons"] > 0]
        return self

    def filter_latest_emissions(self) -> "ClimateDataPipeline":
        """Keeps only the most recent record for each (source_id, gas) based on end_time.

        Returns:
            ClimateDataPipeline: The pipeline instance with only the latest emissions.
        """
        if self.df is not None and {"source_id", "gas", "end_time"}.issubset(
            self.df.columns
        ):
            self.df["end_time"] = pd.to_datetime(self.df["end_time"], errors="coerce")

            self.df["row_no"] = (
                self.df.sort_values(
                    ["source_id", "gas", "end_time"], ascending=[True, True, False]
                )
                .groupby(["source_id", "gas"])
                .cumcount()
                + 1
            )

            self.df = self.df[self.df["row_no"] == 1].drop(columns=["row_no"])
        return self

    def select_final_columns(self) -> "ClimateDataPipeline":
        """Selects only the required columns for final output.

        Returns:
            ClimateDataPipeline: The pipeline instance with selected columns.
        """
        keep_columns: list[str] = [
            "source_id",
            "source_name",
            "lat",
            "lon",
            "source_type",
            "gas",
            "year",
            "original_inventory_sector",
            "average_monthly_emissions_tons",
        ]
        if self.df is not None:
            self.df = self.df[keep_columns]
        return self

    def save_separate_gas_files(self, base_output_path: str) -> None:
        """Saves separate CSV files for each gas type.

        Args:
            base_output_path (str): The base file path without the gas suffix.
        """
        if self.df is not None:
            for gas in self.df["gas"].unique():
                gas_df = self.df[self.df["gas"] == gas]
                gas_output_path = f"{base_output_path}_{gas}.csv"
                gas_df.to_csv(gas_output_path, index=False)

    def run_pipeline(self, output_base_path: str | None = None) -> pd.DataFrame:
        """Runs the full pipeline in sequence and optionally saves separate files for each gas.

        Args:
            output_base_path (str | None): Optional base file path to save the processed DataFrame by gas type.

        Returns:
            pd.DataFrame: Processed DataFrame after running all transformations.
        """
        self.df = (
            self.load_data()
            .convert_dates()
            .filter_gases()
            .calculate_avg_emissions()
            .drop_zero_emissions()
            .filter_latest_emissions()
            .select_final_columns()
            .df
        )

        if output_base_path:
            self.save_separate_gas_files(output_base_path)

        return self.df

In [130]:
climate_pipeline_2023 = ClimateDataPipeline(climate_trace_TX_2023_csv)
final_emissions_data_2023 = climate_pipeline_2023.run_pipeline(
    output_base_path="climate_trace_TX_2023"
)

climate_pipeline_2022 = ClimateDataPipeline(climate_trace_TX_2022_csv)
final_emissions_data_2022 = climate_pipeline_2022.run_pipeline(
    output_base_path="climate_trace_TX_2022"
)

C:\Users\james\AppData\Local\Temp\ipykernel_17652\664249325.py:20: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv(self.file_path)
C:\Users\james\AppData\Local\Temp\ipykernel_17652\664249325.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.df["end_time"] = pd.to_datetime(self.df["end_time"], errors="coerce")
C:\Users\james\AppData\Local\Temp\ipykernel_17652\664249325.py:20: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv(self.file_path)
C:\Users\james\AppData\Local\Temp\ipykernel_17652\664249325.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.

In [1]:
# final_emissions_data_2023

In [2]:
# final_emissions_data_2022

## Emissions comparison

In [160]:
join_tables_query = """
WITH joined_table AS (
    SELECT
        *,    
    FROM final_emissions_data_2023 AS a_2023
    JOIN final_emissions_data_2022 AS a_2022
    ON (a_2023.source_id = a_2022.source_id AND a_2023.gas = a_2022.gas)
)
,

abs_calculation_table AS (   
    SELECT 
        *,
        (average_monthly_emissions_tons - average_monthly_emissions_tons_1) AS emissions_change_abs
    FROM joined_table
)

SELECT
    source_id,
    source_name,
    lat,
    lon,
    source_type,
    gas,
    original_inventory_sector,
    emissions_change_abs AS emissions_change_tons_absolute_vs_2022,
    emissions_change_abs / average_monthly_emissions_tons_1 AS emissions_change_percentage_vs_2022
FROM abs_calculation_table

"""

comparison_df = ddb.query(join_tables_query).to_df()

These are suspicious results... All gases for each source_id seem to change at the exact same rate from 2022 to 2023. I suppose they calculate using the amount produced, and have a baseline value for the amount of gas released for each unit of production. That would explain why the percentages are kept constant. But presumably if they traded one material for another, there would be some discrepancies.

In [162]:
# comparison_df.head(20)

In [ ]:
# for gas in comparison_df["gas"].unique():
#     gas_df = comparison_df[comparison_df["gas"] == gas]
#     gas_output_path = f"emissions_comparison_{gas}.csv"
#     gas_df.to_csv(gas_output_path, index=False)

## Validation

To confirm that the pipeline is transforming my data correctly, I run a validation using DuckDB (to allow me to query the results in SQL). 

### Climate Trace

In [ ]:
# climate_trace_TX_2023_df = pd.read_csv(climate_trace_TX_2023_csv)

C:\Users\james\AppData\Local\Temp\ipykernel_17652\1082219623.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  climate_trace_TX_2023_df = pd.read_csv(climate_trace_TX_2023_csv)


In [136]:
# climate_trace_TX_2023_df.dtypes
# climate_trace_TX_2023_df.describe()
# climate_trace_TX_2023_df.head()

In [ ]:
# climate_trace_TX_2023_df["end_time"] = pd.to_datetime(
#     climate_trace_TX_2023_df["end_time"], errors="coerce"
# )
# # climate_trace_TX_2023_df = climate_trace_TX_2023_df.dropna(subset=["end_time"])
# climate_trace_TX_2023_df["end_time"] = climate_trace_TX_2023_df["end_time"].dt.strftime(
#     "%m-%Y"
# )
# climate_trace_TX_2023_df["year"] = climate_trace_TX_2023_df["end_time"].str[-4:]

In [ ]:
# climate_trace_TX_2023_df["gas"].unique()

array(['bc', 'ch4', 'co', 'co2', 'co2e_100yr', 'co2e_20yr', 'n2o', 'nh3',
       'nmvoc', 'nox', 'oc', 'pm2_5', 'so2'], dtype=object)

In [ ]:
# climate_trace_TX_2023_df = ddb.query(
#     """SELECT * FROM climate_trace_TX_2023_df WHERE gas IN ('ch4', 'co2', 'nox', 'pm2_5') """
# ).to_df()

In [ ]:
# climate_trace_TX_2023_df.columns

Index(['source_id', 'source_name', 'source_type', 'iso3_country',
       'original_inventory_sector', 'start_time', 'end_time',
       'temporal_granularity', 'gas', 'emissions_quantity', 'emissions_factor',
       'emissions_factor_units', 'capacity', 'capacity_units',
       'capacity_factor', 'activity', 'activity_units', 'created_date',
       'modified_date', 'lat', 'lon', 'year'],
      dtype='object')

In [ ]:
# climate_trace_TX_2023_df.describe()

,source_id,emissions_quantity,emissions_factor,capacity,capacity_factor,activity,created_date,modified_date,lat,lon
count,360252.00000,360820.00000,360820.00000,360820.00000,360820.00000,360820.00000,0.00000,0.00000,360252.00000,360252.00000
mean,9584086.64312,2575.71624,74.73219,737177.56238,939.87725,10964551.52903,NaN,NaN,32.40151,-99.16437
std,11734343.42696,55304.17764,2249.03027,12301353.30281,4444.15067,118309744.96233,NaN,NaN,2.33652,2.87028
min,16429.00000,-43775.05833,-0.22323,-0.00000,0.00000,-0.00000,NaN,NaN,25.89800,-106.62500
25%,1621821.00000,0.00000,0.00000,351.00000,0.83400,349.59600,NaN,NaN,30.67278,-102.06099
50%,3172777.00000,0.00000,0.00000,1708.40849,0.99600,2558.34757,NaN,NaN,32.48245,-98.64957
75%,20703827.00000,1.48036,0.00008,28976.91101,1.00000,106448.35000,NaN,NaN,34.32972,-96.89841
max,37699654.00000,5011577.60947,220674.63382,1699021777.82245,125664.21222,7727687489.88827,NaN,NaN,36.50238,-93.56655


Just be aware that there might be multiple emissions sources for a given source name (e.g. Anderson County)

Here I use SQL (using DuckDB) to validate the results I got in Python. Why? Because I am more comfortable in SQL than in Python.

In [ ]:
# avg_emissions_per_month_query = """
# WITH t1 AS (
#     SELECT 
#         *,
#         AVG(emissions_quantity) OVER (PARTITION BY source_id, gas) AS average_monthly_emissions_tons
#     FROM climate_trace_TX_2023_df
# )
# ,

# output AS (
#     SELECT *
#     FROM t1
#     WHERE average_monthly_emissions_tons > 0
# )

# SELECT *
# FROM output
# """

# climate_trace_TX_2023_df = ddb.query(avg_emissions_per_month_query).to_df()

In [ ]:
# climate_trace_TX_2023_df = climate_trace_TX_2023_df.sort_values(
#     by="average_monthly_emissions_tons", ascending=False
# )

# climate_trace_TX_2023_df

,source_id,source_name,source_type,iso3_country,original_inventory_sector,start_time,end_time,temporal_granularity,gas,emissions_quantity,emissions_factor,emissions_factor_units,capacity,capacity_units,capacity_factor,activity,activity_units,created_date,modified_date,lat,lon,year,average_monthly_emissions_tons
173631,3588795.00000,United States_Delaware_Tight oil,Tight oil,USA,oil-and-gas-production,2023-1-1,01-2023,Monthly,co2,5011577.60947,0.00000,t of CO2 per License restricted,0.00000,restricted,0.00000,0.00000,License restricted,NaN,NaN,31.92009,-103.66224,2023,4933597.36126
173630,3588795.00000,United States_Delaware_Tight oil,Tight oil,USA,oil-and-gas-production,2023-2-1,02-2023,Monthly,co2,4869892.55119,0.00000,t of CO2 per License restricted,0.00000,restricted,0.00000,0.00000,License restricted,NaN,NaN,31.92009,-103.66224,2023,4933597.36126
173628,3588795.00000,United States_Delaware_Tight oil,Tight oil,USA,oil-and-gas-production,2023-4-1,04-2023,Monthly,co2,5004011.58152,0.00000,t of CO2 per License restricted,0.00000,restricted,0.00000,0.00000,License restricted,NaN,NaN,31.92009,-103.66224,2023,4933597.36126
173629,3588795.00000,United States_Delaware_Tight oil,Tight oil,USA,oil-and-gas-production,2023-3-1,03-2023,Monthly,co2,4821682.77253,0.00000,t of CO2 per License restricted,0.00000,restricted,0.00000,0.00000,License restricted,NaN,NaN,31.92009,-103.66224,2023,4933597.36126
173623,3588795.00000,United States_Delaware_Tight oil,Tight oil,USA,oil-and-gas-production,2023-9-1,09-2023,Monthly,co2,4810679.12291,0.00000,t of CO2 per License restricted,0.00000,restricted,0.00000,0.00000,License restricted,NaN,NaN,31.92009,-103.66224,2023,4933597.36126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42043,21467175.00000,Bowie County,None,USA,rice-cultivation,2023-7-1,07-2023,Monthly,pm2_5,0.00000,0.00000,t of PM2_5 per harvested ha,0.00000,ha,0.00000,0.00000,harvested ha,NaN,NaN,33.47855,-94.39558,2023,0.00000
42042,21467175.00000,Bowie County,None,USA,rice-cultivation,2023-6-1,06-2023,Monthly,pm2_5,0.00000,0.00000,t of PM2_5 per harvested ha,0.00000,ha,0.00000,0.00000,harvested ha,NaN,NaN,33.47855,-94.39558,2023,0.00000
42036,21467175.00000,Bowie County,None,USA,rice-cultivation,2023-11-1,11-2023,Monthly,pm2_5,0.00000,0.00000,t of PM2_5 per harvested ha,0.00000,ha,0.00000,0.00000,harvested ha,NaN,NaN,33.47855,-94.39558,2023,0.00000
42044,21467175.00000,Bowie County,None,USA,rice-cultivation,2023-8-1,08-2023,Monthly,pm2_5,0.00000,0.00000,t of PM2_5 per harvested ha,0.00000,ha,0.00000,0.00000,harvested ha,NaN,NaN,33.47855,-94.39558,2023,0.00000


In [ ]:
# climate_trace_filtered_query = """
# WITH filtered AS (
#     SELECT
#         source_id,
#         source_name,
#         lat,
#         lon,
#         source_type,
#         gas,
#         year,
#         original_inventory_sector,
#         average_monthly_emissions_tons,
#         ROW_NUMBER() OVER (PARTITION BY source_id, gas ORDER BY end_time DESC) AS row_no
#     FROM climate_trace_TX_2023_df
# )
# ,

# output AS (
#     SELECT *
#     FROM filtered
#     WHERE row_no = 1
# )

# SELECT *
# FROM output

# """

# climate_trace_TX_2023_df = ddb.query(climate_trace_filtered_query).to_df()
# climate_trace_TX_2023_df = climate_trace_TX_2023_df.drop(columns=["row_no"])

In [ ]:
# climate_trace_TX_2023_df

,source_id,source_name,lat,lon,source_type,gas,year,original_inventory_sector,average_monthly_emissions_tons
0,1619412.00000,USA_Texas_MatureDairyCattle_21880,32.20548,-98.36657,manure_managementmaturedairycattle,ch4,2023,manure-management-cattle-operation,29.63797
1,1619433.00000,USA_Texas_MatureDairyCattle_21901,34.90731,-102.42430,manure_managementmaturedairycattle,ch4,2023,manure-management-cattle-operation,32.52281
2,1619434.00000,USA_Texas_MatureDairyCattle_21902,33.05819,-95.50478,manure_managementmaturedairycattle,nox,2023,manure-management-cattle-operation,0.06035
3,1619438.00000,USA_Texas_MatureDairyCattle_21906,31.89639,-98.37417,manure_managementmaturedairycattle,ch4,2023,manure-management-cattle-operation,2.36985
4,1619439.00000,USA_Texas_MatureDairyCattle_21907,33.08231,-95.47215,manure_managementmaturedairycattle,nox,2023,manure-management-cattle-operation,0.06035
...,...,...,...,...,...,...,...,...,...
14888,33356617.00000,USA_Texas_MatureDairyCattle_21602,33.06090,-95.53502,enteric_fermentation_maturedairycattle,ch4,2023,enteric-fermentation-cattle-operation,3.72903
14889,33363549.00000,USA_Texas_OtherBeefCattle_21008,29.84948,-97.67096,enteric_fermentation_otherbeefcattle,ch4,2023,enteric-fermentation-cattle-operation,0.32051
14890,33998965.00000,USA-Harris_methanol,29.85069,-95.38101,methanol,nox,2023,chemicals,143.83512
14891,33999103.00000,USA-Hutchinson_ammonia,35.84013,-101.35402,ammonia,nox,2023,chemicals,44.18690


# Appendix

In [146]:
# unep_methane_data_csv = os.path.join(
#     base_folder,
#     "UNEP",
#     "unep_methanedata_detected_plumes_csv",
#     "unep_methanedata_detected_plumes.csv",
# )

# carbon_mapper_plumes_csv = os.path.join(
#     base_folder, "Carbon Mapper", "plumes_2025-03-14T07_39_08.777Z.csv"
# )

# carbon_mapper_sources_json = os.path.join(
#     base_folder, "Carbon Mapper", "sources_2025-03-14T07_38_48.851Z.json"
# )

### Carbon Mapper

In [147]:
# cm_plumes_df = ddb.read_csv(carbon_mapper_plumes_csv)

# with open(carbon_mapper_sources_json, "r") as file:
#     data = json.load(file)

# features = data["features"]

# # Flatten the JSON structure
# cm_sources_df = pd.json_normalize(features)

In [148]:
# cm_sources_df.columns = cm_sources_df.columns.str.split(".").str[-1]
# cm_sources_df.rename(columns={"emission_auto": "emission_auto_kg_per_hr"}, inplace=True)
# cm_sources_df.columns

In [149]:
# cm_sources_df["year"] = pd.to_datetime(cm_sources_df["timestamp_max"]).dt.year

Filter all data prior to 2023. (If a source is first measured in 2022 but last measured in 2023, I include it as part of the 2023 data.)

In [150]:
# cm_sources_df = ddb.query("""SELECT * FROM cm_sources_df WHERE year >= 2023""").to_df()

In [151]:
# cm_sources_df.describe()

I convert the Geo JSON file to CSV.

In [152]:
# cm_sources_df.to_csv("sources_2025-03-14T07_38_48.851Z.csv", index=False)

### UNEP

In [153]:
# unep_methane_data_df = pd.read_csv(unep_methane_data_csv)

In [154]:
# unep_methane_data_df["country"].unique()

In [155]:
# filter_unep_query = """
# SELECT *
# FROM unep_methane_data_df
# WHERE country LIKE 'United States%'
# """

# unep_methane_data_df = ddb.query(filter_unep_query).to_df()

In [156]:
# unep_methane_data_df.head()